In [2]:
import json
import os
import _jsonnet
import os
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

In [3]:
from seq2struct.commands.infer import Inferer
from seq2struct.datasets.spider import SpiderItem
from seq2struct.utils import registry

In [4]:
import torch

In [5]:
# Define a function to load JSON data from a file
def load_json(filename):
    # Open the specified file in read mode, specifying utf-8 encoding
    with open(filename, 'r', encoding='utf-8') as file:
        # Load the JSON data from the file into a Python dictionary
        data = json.load(file)
    # Return the loaded data
    return data

In [6]:
# exp_config = json.loads(_jsonnet.evaluate_file("experiments/spider-configs/gap-run.jsonnet"))
exp_config = json.loads(_jsonnet.evaluate_file("experiments/spider-configs/spider-mT5-base-en-train_en_eval.jsonnet"))

In [7]:
model_config_path = exp_config["model_config"]
model_config_args = exp_config.get("model_config_args")

#print(model_config_path)
#print(model_config_args)

In [8]:
def createPath(lr, bert_lr, end_lr, bs, att):
    lr_s = '%0.1e' % lr
    bert_lr_s = '%0.1e' % bert_lr
    end_lr_s = '0e0' if end_lr == 0 else '%0.1e' % end_lr

    base_bert_enc_size = 768
    enc_size = base_bert_enc_size

    model_name = '/bs=%(bs)d,lr=%(lr)s,bert_lr=%(bert_lr)s,end_lr=%(end_lr)s,att=%(att)d' % {
        'bs': bs,
        'lr': lr_s,
        'bert_lr': bert_lr_s,
        'end_lr': end_lr_s,
        'att': att
    }
    return model_name

In [9]:
infer_config = json.loads(
    _jsonnet.evaluate_file(
        model_config_path, 
        tla_codes={'args': json.dumps(model_config_args)}))

#print(infer_config['model']['encoder_preproc'])

In [10]:
db_path = infer_config["model"]["encoder_preproc"]["db_path"]

db_path_for_json = infer_config['data']['val']['tables_paths'][0]

In [11]:
db_path_for_json

'data/spider-en/tables.json'

In [12]:
schemas = os.listdir(db_path)

schemas_list = load_json(db_path_for_json)

In [13]:
import random
N=len(schemas)

selected_schemas = random.choices(schemas, k=N)
print('You can choose a database from the below:')
print(', '.join(selected_schemas))

You can choose a database from the below:
activity_1, wrestler, club_1, department_store, e_government, singer, architecture, film_rank, culture_company, course_teach, architecture, car_1, pilot_record, entrepreneur, college_3, csu_1, ship_mission, debate, loan_1, customer_deliveries, company_employee, program_share, climbing, music_1, voter_2, gas_company, cre_Doc_Control_Systems, scientist_1, party_host, voter_1, customers_campaigns_ecommerce, election, aircraft, entrepreneur, insurance_and_eClaims, cre_Doc_Tracking_DB, csu_1, hospital_1, orchestra, customers_and_products_contacts, gymnast, cre_Docs_and_Epenses, voter_1, voter_2, restaurants, ship_1, epinions_1, coffee_shop, restaurants, scientist_1, customer_deliveries, company_1, voter_2, product_catalog, ship_1, station_weather, customers_campaigns_ecommerce, singer, customers_and_addresses, chinook_1, sports_competition, party_people, device, theme_gallery, movie_1, voter_2, concert_singer, customers_campaigns_ecommerce, behavior

In [14]:
inferer = Inferer(infer_config)

WARNING <class 'seq2struct.models.enc_dec.EncDecModel.Preproc'>: superfluous {'name': 'EncDec'}


SpiderEncoderT5Preproc Model: google/mt5-base
No GAP - Generation-Augmented Pre-Training
T5 load Model: google/mt5-base


In [15]:
# model_dir = exp_config["logdir"] + "/bs=12,lr=1.0e-04,bert_lr=1.0e-05,end_lr=0e0,att=1"
# checkpoint_step = exp_config["eval_steps"][0]

# '/home/studio-lab-user/gr-spider/Database-Systems/gap-text2sql/mrat-sql-gap/logdir/mT5-base-en-train/bs=4,lr=1.0e-04,bert_lr=1.0e-05,end_lr=0e0,att=1/val_loss.txt'
logdir_path = exp_config["logdir"]
path_save = createPath(model_config_args['lr'], model_config_args['bert_lr'], model_config_args['end_lr'], model_config_args['bs'], model_config_args['att'])
model_dir = logdir_path + path_save

checkpoint_step = exp_config["eval_steps"][0]

In [16]:
model = inferer.load_model(model_dir, checkpoint_step)

WARNING <class 'seq2struct.models.enc_dec.EncDecModel'>: superfluous {'decoder_preproc': {'grammar': {'clause_order': None, 'end_with_from': True, 'factorize_sketch': 2, 'include_literals': False, 'infer_from_conditions': True, 'name': 'spider', 'output_from': True, 'use_table_pointer': True}, 'save_path': 'data/spider-en/mT5-base-nl2code-1115,output_from=true,fs=2,emb=t5,cvlink', 'use_seq_elem_rules': True}, 'encoder_preproc': {'compute_cv_link': True, 'compute_sc_link': True, 'db_path': 'data/spider-en/database', 'fix_issue_16_primary_keys': True, 'include_table_name_in_column': False, 'pretrained_checkpoint': 'models/mt5-base/pretrained_checkpoint/pytorch_model.bin', 'save_path': 'data/spider-en/mT5-base-nl2code-1115,output_from=true,fs=2,emb=t5,cvlink', 't5_version': 'google/mt5-base'}}


SpiderEncoderT5 Model: google/mt5-base
No GAP - Generation-Augmented Pre-Training


Some weights of the model checkpoint at google/mt5-base were not used when initializing MT5Model: ['lm_head.weight']
- This IS expected if you are initializing MT5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MT5Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Parameter containing:
tensor([[ 1.7500e+00, -1.6719e+00,  2.4062e+00,  ...,  6.9580e-03,
         -9.8828e-01, -4.6875e-01],
        [ 8.5625e+00,  5.5625e+00, -1.7109e+00,  ...,  7.7812e+00,
         -5.2812e+00, -3.2188e+00],
        [ 6.8750e-01, -4.5312e-01,  5.7812e-01,  ...,  7.3828e-01,
         -3.0078e-01,  2.0312e-01],
        ...,
        [ 8.0859e-01, -1.2812e+00,  2.2344e+00,  ...,  8.0469e-01,
          7.7344e-01,  1.8672e+00],
        [ 2.9688e-01, -1.4453e+00,  9.6484e-01,  ...,  8.9453e-01,
          2.3906e+00, -4.1797e-01],
        [ 1.0469e+00, -2.7930e-01,  9.7266e-01,  ..., -3.1250e+00,
          1.2344e+00, -7.6562e-01]], requires_grad=True)
Parameter containing:
tensor([[ 1.7500e+00, -1.6719e+00,  2.4062e+00,  ...,  6.9580e-03,
         -9.8828e-01, -4.6875e-01],
        [ 8.5625e+00,  5.5625e+00, -1.7109e+00,  ...,  7.7812e+00,
         -5.2812e+00, -3.2188e+00],
        [ 6.8750e-01, -4.5312e-01,  5.7812e-01,  ...,  7.3828e-01,
         -3.0078e-01,  2.0312e-

In [17]:
from seq2struct.datasets.spider_lib.preprocess.get_tables import dump_db_json_schema
from seq2struct.datasets.spider import load_tables_from_schema_dict

In [18]:
db_id = 'loan_1'

In [19]:
def get_schema_dict_by_db_id(list_of_schemas_dicts, db_id):
    return [schema_info for schema_info in list_of_schemas_dicts if db_id in schema_info['db_id']][0]

In [20]:
#selected_schema = dump_db_json_schema("{path}/{db_id}/{db_id}.sqlite".format(path=db_path, db_id=db_id), db_id)
selected_schema = get_schema_dict_by_db_id(schemas_list, db_id)

In [21]:
from seq2struct.utils.api_utils import refine_schema_names
# If you want to change your schema name, then run this; Otherwise you can skip this.
#refine_schema_names(my_schema)

In [22]:
selected_schema

{'column_names': [[-1, '*'],
  [0, 'branch id'],
  [0, 'bname'],
  [0, 'no of customers'],
  [0, 'city'],
  [0, 'state'],
  [1, 'customer id'],
  [1, 'customer name'],
  [1, 'account type'],
  [1, 'account balance'],
  [1, 'number of loans'],
  [1, 'credit score'],
  [1, 'branch id'],
  [1, 'state'],
  [2, 'loan id'],
  [2, 'loan type'],
  [2, 'customer id'],
  [2, 'branch id'],
  [2, 'amount']],
 'column_names_original': [[-1, '*'],
  [0, 'branch_ID'],
  [0, 'bname'],
  [0, 'no_of_customers'],
  [0, 'city'],
  [0, 'state'],
  [1, 'cust_ID'],
  [1, 'cust_name'],
  [1, 'acc_type'],
  [1, 'acc_bal'],
  [1, 'no_of_loans'],
  [1, 'credit_score'],
  [1, 'branch_ID'],
  [1, 'state'],
  [2, 'loan_ID'],
  [2, 'loan_type'],
  [2, 'cust_ID'],
  [2, 'branch_ID'],
  [2, 'amount']],
 'column_types': ['text',
  'number',
  'text',
  'number',
  'text',
  'text',
  'text',
  'text',
  'text',
  'number',
  'number',
  'number',
  'number',
  'text',
  'text',
  'text',
  'text',
  'text',
  'number']

In [23]:
def create_dfs_from_schema(schema):
    # Create DataFrame for table names
    table_names_df = pd.DataFrame({
        'table_names_original': schema['table_names_original'],
        'table_names': schema['table_names']
    })

    # Map table indices to table names
    table_index_to_name = {index: name for index, name in enumerate(schema['table_names'])}
    
    # Create DataFrame for column names with table names
    column_data = []
    for col_index, (table_index, col_name) in enumerate(schema['column_names']):
        table_name = table_index_to_name[table_index] if table_index != -1 else 'N/A'
        is_primary_key = col_index in schema['primary_keys']
        column_data.append((col_index, table_name, schema['column_names_original'][col_index][1], col_name, schema['column_types'][col_index], is_primary_key))

    column_names_df = pd.DataFrame(column_data, columns=['column_index', 'table_name', 'column_names_original', 'column_names', 'column_types', 'is_primary_key'])
    
    column_index_to_name = [name for index, name in schema['column_names']]

    # Create DataFrame for primary keys
    primary_keys_df = pd.DataFrame({
    'table_name': list(table_index_to_name.values()),
    'primary_key_column_index': schema['primary_keys'],
    'primary_key': [column_index_to_name[x] for x in schema['primary_keys']]
    })

    
    column_index_to_id = {index: name for index, name in enumerate(schema['column_names'])}
    
    foreign_keys_df = pd.DataFrame({
    'foreign_key_column': [x[0] for x in schema['foreign_keys']],
    'referenced_column': [x[1] for x in schema['foreign_keys']],
    'foreign_key_table': [table_index_to_name[column_index_to_id[x[0]][0]] for x in schema['foreign_keys']],
    'foreign_key_column_name': [column_index_to_id[x[0]][1] for x in schema['foreign_keys']],
    'referenced_table': [table_index_to_name[column_index_to_id[x[1]][0]] for x in schema['foreign_keys']],
    'referenced_column_name': [column_index_to_id[x[1]][1] for x in schema['foreign_keys']]
    }) 

    return table_names_df, column_names_df, primary_keys_df, foreign_keys_df

In [24]:
table_names_df, column_names_df, primary_keys_df, foreign_keys_df = create_dfs_from_schema(selected_schema)

In [25]:
table_names_df

,table_names_original,table_names
0,bank,bank
1,customer,customer
2,loan,loan


In [26]:
column_names_df

,column_index,table_name,column_names_original,column_names,column_types,is_primary_key
0,0,N/A,*,*,text,False
1,1,bank,branch_ID,branch id,number,True
2,2,bank,bname,bname,text,False
3,3,bank,no_of_customers,no of customers,number,False
4,4,bank,city,city,text,False
5,5,bank,state,state,text,False
6,6,customer,cust_ID,customer id,text,True
7,7,customer,cust_name,customer name,text,False
8,8,customer,acc_type,account type,text,False
9,9,customer,acc_bal,account balance,number,False


In [27]:
primary_keys_df

,table_name,primary_key_column_index,primary_key
0,bank,1,branch id
1,customer,6,customer id
2,loan,14,loan id


In [28]:
foreign_keys_df

,foreign_key_column,referenced_column,foreign_key_table,foreign_key_column_name,referenced_table,referenced_column_name
0,12,1,customer,branch id,bank,branch id
1,17,1,loan,branch id,bank,branch id


In [29]:
schema, eval_foreign_key_maps = load_tables_from_schema_dict(selected_schema)
#print(schema.keys())
#print(eval_foreign_key_maps) 

In [30]:
dataset = registry.construct('dataset_infer', {
   "name": "spider", "schemas": schema, "eval_foreign_key_maps": eval_foreign_key_maps, 
    "db_path": db_path
})

In [31]:
for _, schema_info in dataset.schemas.items():
    model.preproc.enc_preproc._preprocess_schema(schema_info)

In [32]:
spider_schema = dataset.schemas[db_id]

In [33]:
def infer(question):
    data_item = SpiderItem(
            text=None,  # intentionally None -- should be ignored when the tokenizer is set correctly
            code=None,
            schema=spider_schema,
            orig_schema=spider_schema.orig,
            orig={"question": question}
        )
    model.preproc.clear_items()
    enc_input = model.preproc.enc_preproc.preprocess_item(data_item, None)
    preproc_data = enc_input, None
    with torch.no_grad():
        output = inferer._infer_one(model, data_item, preproc_data, beam_size=1, use_heuristic=True)
    return output[0]["inferred_code"]

In [34]:
code = infer("Which branch has given the highest number loans?")
print(code)

SELECT loan.branch_ID FROM bank JOIN loan GROUP BY loan.branch_ID ORDER BY Count(*) Desc LIMIT 1
